In [65]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.nn.functional as F


In [66]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

device

device(type='mps')

In [67]:
anime_data = pd.read_csv('./Anime Dataset 2023/anime-dataset-2023.csv', on_bad_lines='skip', encoding='utf-8')
user_data = pd.read_csv('./Anime Dataset 2023/users-details-2023.csv', on_bad_lines='skip', encoding='utf-8')
user_scores = pd.read_csv('./Anime Dataset 2023/users-score-2023.csv', on_bad_lines='skip', encoding='utf-8')

In [68]:
anime_data.head()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


In [69]:
user_data.head()

,Mal ID,Username,Gender,Birthday,Location,Joined,Days Watched,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes Watched
0,1,Xinil,Male,1985-03-04T00:00:00+00:00,California,2004-11-05T00:00:00+00:00,142.3,7.37,1.0,233.0,8.0,93.0,64.0,399.0,60.0,8458.0
1,3,Aokaado,Male,NaN,"Oslo, Norway",2004-11-11T00:00:00+00:00,68.6,7.34,23.0,137.0,99.0,44.0,40.0,343.0,15.0,4072.0
2,4,Crystal,Female,NaN,"Melbourne, Australia",2004-11-13T00:00:00+00:00,212.8,6.68,16.0,636.0,303.0,0.0,45.0,1000.0,10.0,12781.0
3,9,Arcane,NaN,NaN,NaN,2004-12-05T00:00:00+00:00,30.0,7.71,5.0,54.0,4.0,3.0,0.0,66.0,0.0,1817.0
4,18,Mad,NaN,NaN,NaN,2005-01-03T00:00:00+00:00,52.0,6.27,1.0,114.0,10.0,5.0,23.0,153.0,42.0,3038.0


In [70]:
user_scores.head()

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7
2,1,Xinil,320,A Kite,5
3,1,Xinil,49,Aa! Megami-sama!,8
4,1,Xinil,304,Aa! Megami-sama! Movie,8


In [71]:
data = pd.merge(user_scores, user_data, left_on='user_id', right_on='Mal ID')
data = pd.merge(data, anime_data, on='anime_id')

In [72]:
data.head()

,user_id,Username_x,anime_id,Anime Title,rating,Mal ID,Username_y,Gender,Birthday,Location,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Xinil,21,One Piece,9,1,Xinil,Male,1985-03-04T00:00:00+00:00,California,...,Toei Animation,Manga,24 min,PG-13 - Teens 13 or older,55.0,20,198986,1226493.0,2168904,https://cdn.myanimelist.net/images/anime/6/732...
1,1,Xinil,48,.hack//Sign,7,1,Xinil,Male,1985-03-04T00:00:00+00:00,California,...,Bee Train,Original,24 min per ep,PG-13 - Teens 13 or older,4172.0,1243,1343,83009.0,178659,https://cdn.myanimelist.net/images/anime/1443/...
2,1,Xinil,320,A Kite,5,1,Xinil,Male,1985-03-04T00:00:00+00:00,California,...,Arms,Original,26 min per ep,Rx - Hentai,UNKNOWN,2485,302,33411.0,66821,https://cdn.myanimelist.net/images/anime/1224/...
3,1,Xinil,49,Aa! Megami-sama!,8,1,Xinil,Male,1985-03-04T00:00:00+00:00,California,...,AIC,Manga,30 min per ep,PG-13 - Teens 13 or older,2542.0,2832,246,26400.0,52627,https://cdn.myanimelist.net/images/anime/11/71...
4,1,Xinil,304,Aa! Megami-sama! Movie,8,1,Xinil,Male,1985-03-04T00:00:00+00:00,California,...,AIC,Original,1 hr 46 min,PG-13 - Teens 13 or older,1590.0,3195,99,22479.0,42124,https://cdn.myanimelist.net/images/anime/5/712...


In [73]:
len(data)

23803248

In [74]:
popular_anime = data['anime_id'].value_counts()
popular_anime_ids = popular_anime[popular_anime > 500].index
data = data[data['anime_id'].isin(popular_anime_ids)]

In [75]:
active_users = data['user_id'].value_counts()
active_user_ids = active_users[active_users > 100].index
data = data[data['user_id'].isin(active_user_ids)]

In [76]:
len(data)

17870130

In [77]:
data.isnull().sum()

user_id                   0
Username_x              205
anime_id                  0
Anime Title               0
rating                    0
Mal ID                    0
Username_y              205
Gender              4231557
Birthday            8567127
Location            8088077
Joined                    0
Days Watched            302
Mean Score              302
Watching                302
Completed               302
On Hold                 302
Dropped                 302
Plan to Watch           302
Total Entries           302
Rewatched               302
Episodes Watched        302
Name                      0
English name              0
Other name                0
Score                     0
Genres                    0
Synopsis                  0
Type                      0
Episodes                  0
Aired                     0
Premiered                 0
Status                    0
Producers                 0
Licensors                 0
Studios                   0
Source              

In [78]:

data = data.drop(columns=['Gender', 'Birthday', 'Location', 'Username_x', 'Username_y', 'Mal ID', 'Anime Title', 'Name', 'English name', 'Other name', 'Synopsis', 'Aired',
    'Premiered', 'Producers', 'Licensors', 'Studios', 'Source', 'Image URL',
    'Joined', 'Status', 'Duration', 'Rank', 'Scored By'])


In [79]:
num_cols = ['Days Watched', 'Mean Score', 'Watching', 'Completed',
            'On Hold', 'Dropped', 'Plan to Watch', 'Total Entries',
            'Rewatched', 'Episodes Watched']

for col in num_cols:
    data[col].fillna(data[col].mean(), inplace=True)


/var/folders/8y/qyq16t6549scb0wwphs6rhjm0000gn/T/ipykernel_9688/2019104120.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mean(), inplace=True)


In [80]:
data.isnull().sum()


user_id             0
anime_id            0
rating              0
Days Watched        0
Mean Score          0
Watching            0
Completed           0
On Hold             0
Dropped             0
Plan to Watch       0
Total Entries       0
Rewatched           0
Episodes Watched    0
Score               0
Genres              0
Type                0
Episodes            0
Rating              0
Popularity          0
Favorites           0
Members             0
dtype: int64

In [81]:
data.head()

,user_id,anime_id,rating,Days Watched,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,...,Rewatched,Episodes Watched,Score,Genres,Type,Episodes,Rating,Popularity,Favorites,Members
0,1,21,9,142.3,7.37,1.0,233.0,8.0,93.0,64.0,...,60.0,8458.0,8.69,"Action, Adventure, Fantasy",TV,UNKNOWN,PG-13 - Teens 13 or older,20,198986,2168904
1,1,48,7,142.3,7.37,1.0,233.0,8.0,93.0,64.0,...,60.0,8458.0,6.95,"Adventure, Fantasy, Mystery",TV,26.0,PG-13 - Teens 13 or older,1243,1343,178659
2,1,320,5,142.3,7.37,1.0,233.0,8.0,93.0,64.0,...,60.0,8458.0,6.54,"Action, Drama, Hentai",OVA,2.0,Rx - Hentai,2485,302,66821
3,1,49,8,142.3,7.37,1.0,233.0,8.0,93.0,64.0,...,60.0,8458.0,7.29,"Comedy, Romance, Supernatural",OVA,5.0,PG-13 - Teens 13 or older,2832,246,52627
4,1,304,8,142.3,7.37,1.0,233.0,8.0,93.0,64.0,...,60.0,8458.0,7.54,"Comedy, Romance, Supernatural",Movie,1.0,PG-13 - Teens 13 or older,3195,99,42124


In [82]:

data_sampled, _ = train_test_split(
    data,
    train_size=0.5,
    stratify=data['rating'],
    random_state=42
)


In [83]:
len(data_sampled)

8935065

In [84]:
data_sampled.columns

Index(['user_id', 'anime_id', 'rating', 'Days Watched', 'Mean Score',
       'Watching', 'Completed', 'On Hold', 'Dropped', 'Plan to Watch',
       'Total Entries', 'Rewatched', 'Episodes Watched', 'Score', 'Genres',
       'Type', 'Episodes', 'Rating', 'Popularity', 'Favorites', 'Members'],
      dtype='object')

In [85]:
numeric_features = [
    'user_id', 'anime_id', 'Days Watched', 'Mean Score', 'Watching',
    'Completed', 'On Hold', 'Dropped', 'Plan to Watch', 'Total Entries',
    'Rewatched', 'Episodes Watched', 'Popularity', 'Favorites', 'Members'
]


In [87]:
scaler = StandardScaler()
X_numeric = scaler.fit_transform(data_sampled[numeric_features])

X = data_sampled[['user_id', 'anime_id']].values
X = np.hstack((X, X_numeric))
y = pd.to_numeric(data_sampled['rating'], errors='coerce').fillna(0).astype(float).values


X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(data_sampled['rating'].values, dtype=torch.float32).unsqueeze(1)



In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
data_sampled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8935065 entries, 22637650 to 13277692
Data columns (total 21 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           int64  
 1   anime_id          int64  
 2   rating            int64  
 3   Days Watched      float64
 4   Mean Score        float64
 5   Watching          float64
 6   Completed         float64
 7   On Hold           float64
 8   Dropped           float64
 9   Plan to Watch     float64
 10  Total Entries     float64
 11  Rewatched         float64
 12  Episodes Watched  float64
 13  Score             object 
 14  Genres            object 
 15  Type              object 
 16  Episodes          object 
 17  Rating            object 
 18  Popularity        int64  
 19  Favorites         int64  
 20  Members           int64  
dtypes: float64(10), int64(6), object(5)
memory usage: 1.5+ GB


In [90]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [91]:
class WideAndDeep(nn.Module):
    def __init__(self, n_users, n_animes, embed_dim):
        super(WideAndDeep, self).__init__()
        self.user_embed = nn.Embedding(n_users, embed_dim)
        self.anime_embed = nn.Embedding(n_animes, embed_dim)
        self.wide = nn.Linear(15, 1)
        self.deep = nn.Sequential(
            nn.Linear(embed_dim * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, user_id, anime_id, features):
        wide_out = self.wide(features)
        user_embedding = self.user_embed(user_id)
        anime_embedding = self.anime_embed(anime_id)
        deep_input = torch.cat([user_embedding, anime_embedding], dim=1)
        deep_out = self.deep(deep_input)
        output = wide_out + deep_out
        return F.sigmoid(output) * 10

In [92]:
n_users = data_sampled['user_id'].nunique()
n_animes = data_sampled['anime_id'].nunique()
embed_dim = 32

model = WideAndDeep(n_users, n_animes, embed_dim).to(device)
# criterion = nn.MSELoss()
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)



In [94]:
epochs = 10
best_loss = float('inf')

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        user_id = X_batch[:, 0].long()
        anime_id = X_batch[:, 1].long()
        features = X_batch[:, 2:]

        optimizer.zero_grad()
        predictions = model(user_id, anime_id, features)
        loss = criterion(predictions, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    epoch_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), 'wide_and_deep_model_weights.pth')
        print("Model weights saved.")

print("Training completed.")


Epoch 1, Loss: 1.0283
Model weights saved.
Epoch 2, Loss: 1.0228
Model weights saved.
Epoch 3, Loss: 1.0213
Model weights saved.
Epoch 4, Loss: 1.0202
Model weights saved.
Epoch 5, Loss: 1.0192
Model weights saved.
Epoch 6, Loss: 1.0182
Model weights saved.
Epoch 7, Loss: 1.0173
Model weights saved.
Epoch 8, Loss: 1.0165
Model weights saved.
Epoch 9, Loss: 1.0157
Model weights saved.
Epoch 10, Loss: 1.0149
Model weights saved.
Training completed.


In [95]:
model.eval()
total_loss = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        user_id = X_batch[:, 0].long()
        anime_id = X_batch[:, 1].long()
        features = X_batch[:, 2:]

        predictions = model(user_id, anime_id, features)
        loss = criterion(predictions, y_batch)
        total_loss += loss.item()

print(f"Test Loss: {total_loss / len(test_loader):.4f}")


Test Loss: 1.0193


In [96]:
def rmse(predictions, targets):
    return torch.sqrt(((predictions - targets) ** 2).mean())

def calculate_accuracy(predictions, targets, tolerance=1.0):
    correct = (torch.abs(predictions - targets) <= tolerance).float().sum()
    return correct / targets.size(0)


with torch.no_grad():
    all_predictions, all_targets = [], []
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        user_id = X_batch[:, 0].long()
        anime_id = X_batch[:, 1].long()
        features = X_batch[:, 2:]

        preds = model(user_id, anime_id, features).squeeze()
        all_predictions.append(preds)
        all_targets.append(y_batch.squeeze())

    all_predictions = torch.cat(all_predictions)
    all_targets = torch.cat(all_targets)

    final_rmse = rmse(all_predictions, all_targets)
    accuracy = calculate_accuracy(all_predictions, all_targets, tolerance=1.0)

print(f"Test RMSE: {final_rmse:.4f}")
print(f"Accuracy (±1 tolerance): {accuracy:.2%}")


Test RMSE: 1.3431
Accuracy (±1 tolerance): 59.06%


: 